# 0. Load all dependencies

In [29]:

# Importing some useful packages

from importlib import reload
from matplotlib.patches import Polygon
from moviepy.editor import VideoFileClip
from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import sys
import os
import glob
import pickle
import cv2

sys.path.append("../../") # go to parent dir

# Visualizations will be shown in the notebook:

%matplotlib inline


# Import local deps

import src.helpers.lineRegressorWithMemory as LRM
import src.helpers.imageProcessing as IP
import src.helpers.plot as P
import src.helpers.cameraCalibration as C
import src.helpers.laneFinder as LF
import src.helpers.imagePreprocessingPipeline as IPP
import src.helpers.polyFitPipeline as PFP


if 'src.helpers.imageProcessing' in sys.modules:
    print("Reloading Modules...")
    
    reload(LRM)
    reload(IP)
    reload(P)
    reload(C)
    reload(LF)
    reload(IPP)
    reload(PFP)

    print("\nModules Reloaded!")


Reloading Modules...

Modules Reloaded!


# 1. Load camera calibration and calculate M transform matrix

In [22]:
# Load an image

anyImage = mpimg.imread("../../input/images/001 - Yellow Straight.jpg")


# Get image count & dimensions

WIDTH = anyImage.shape[1]
HEIGHT = anyImage.shape[0]


# Load camera calibration

data = pickle.load(open('../../output/data/calibration.p', 'rb'))
mtx = data["mtx"]
dist = data["dist"]


# Calculate perpective transform matrixs

MIDDLE = WIDTH // 2
H_MARGIN_TOP = 50
H_MARGIN_BOTTOM = 215
CORRECTION = 15
DST_MARGIN_H = 400
DST_MARGIN_TOP = 100

src = np.array([
    (MIDDLE - H_MARGIN_TOP + CORRECTION - 10, 450),
    (MIDDLE + H_MARGIN_TOP + CORRECTION - 22, 450), 
    (WIDTH - H_MARGIN_BOTTOM + CORRECTION - 5, 700), 
    (H_MARGIN_BOTTOM + CORRECTION, 700)
], np.float32)

dst = np.array([
    (DST_MARGIN_H, DST_MARGIN_TOP),
    (WIDTH - DST_MARGIN_H, DST_MARGIN_TOP),
    (WIDTH - DST_MARGIN_H, HEIGHT),
    (DST_MARGIN_H, HEIGHT)
], np.float32)

M, Minv = C.getM(src, dst)


# 2. Create video processing pipeline

In [23]:

# IMAGE PROCESSING:

# rightLineStream = LRM.LineStream()
# leftLineStream = LRM.LineStream()

def process_image(imageRGB):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    imgUndistorted = C.undistort(imageRGB, mtx, dist)
    imgBinary = IPP.pipeline(imgUndistorted, M)
    
    return PFP.pipeline(imgUndistorted, imgBinary, Minv)


# VIDEO PROCESSING:

outputDir = "../../output/videos/"
videosDir = "../../input/videos/"
videosNames = os.listdir(videosDir)

videosNames = ["004 - Advanced Project.mp4"]

def process_video(videoName):
    print("--------\n\nProcessing %s:" % videoName)

    # rightLineStream.reset()
    # leftLineStream.reset()
    
    inputClip = VideoFileClip(videosDir + videoName)
    outputClip = inputClip.fl_image(process_image) # NOTE: this function expects color images!
    %time outputClip.write_videofile(outputDir + videoName, audio=False)
    
    return outputClip
    
print("Available videos:\n")

for videoName in videosNames: print(videoName)
    

Available videos:

004 - Advanced Project.mp4


# 3. Process videos

In [30]:

for videoName in videosNames:
    output = process_video(videoName)


--------

Processing 004 - Advanced Project.mp4:
[MoviePy] >>>> Building video ../../output/videos/004 - Advanced Project.mp4
[MoviePy] Writing video ../../output/videos/004 - Advanced Project.mp4


100%|██████████████████████████████████████████████████████████████████████████▉| 1260/1261 [04:11<00:00,  4.79it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../../output/videos/004 - Advanced Project.mp4 

Wall time: 4min 11s
